In [ ]:
%pip install gradio
%pip install --upgrade --quiet  duckduckgo-search
%pip install langchain_community
%pip install langchainhub
%pip install langchain-groq
%pip install wikipedia


In [ ]:
import gradio as gr
from langchain_community.tools.google_finance import GoogleFinanceQueryRun
from langchain_community.utilities.google_finance import GoogleFinanceAPIWrapper
from langchain_community.utilities import SerpAPIWrapper
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.tools import Tool
from langchain_groq import ChatGroq
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_community.utilities.alpha_vantage import AlphaVantageAPIWrapper
from wikipedia import summary

In [ ]:
def search_wikipedia(query):
    """Searches Wikipedia and returns the summary of the first result."""
    try:
        # Limit to two sentences for brevity
        return summary(query, sentences=2)
    except:
        return "I couldn't find any information on that."
# Create functions for Alpha Vantage tools
# Initialize Alpha Vantage wrapper
alphavantage_api_key=""
alpha_vantage = AlphaVantageAPIWrapper(alphavantage_api_key=alphavantage_api_key)
def get_exchange_rate(query: str) -> str:
    """Get the exchange rate between two currencies."""
    from_currency, to_currency = query.split(',')
    return alpha_vantage._get_exchange_rate(from_currency.strip(), to_currency.strip())

def get_stock_quote(symbol: str) -> str:
    """Get the current stock quote for a given symbol."""
    return alpha_vantage._get_quote_endpoint(symbol)
def ask_google(message,memory):
    try:
        
        serp_api_key=""
        google_finance_tool = Tool(
            name="GoogleFinance",
            func=GoogleFinanceQueryRun(api_wrapper=GoogleFinanceAPIWrapper(serp_api_key=serp_api_key)),
            description="Useful for financial queries and stock market data."
        )
        params = {
            "engine": "google",
            "gl": "us",
            "hl": "en",
        }
        tools = [
            google_finance_tool,
            Tool(
                name="Exchange Rate",
                func=get_exchange_rate,
                description="Use this tool to get the exchange rate between two currencies. Input should be two comma-separated currency codes, e.g., 'USD,EUR'."
            ),
            Tool(
                name="Stock Quote",
                func=get_stock_quote,
                description="Use this tool to get the current stock quote for a given symbol. Input should be a single stock symbol, e.g., 'AAPL'."
            ),
            Tool(
                name="SerpAPI",
                func=SerpAPIWrapper(params=params,serpapi_api_key=serp_api_key).run,
                description="Useful for retrieving search engine results and performing web searches.",
            ),
            Tool(
                name="DuckDuckGo Search",
                func=DuckDuckGoSearchRun(),
                description="Useful for privacy-focused web searches and retrieving diverse search results.",
            ),
            Tool(
            name="Wikipedia",
            func=search_wikipedia,
            description="Useful for when you need to know information about a topic.",
        ),
        ]
        api_key=""
        llm= ChatGroq(temperature=0, groq_api_key=api_key, model_name='mixtral-8x7b-32768' )
        memory = ConversationBufferMemory(
            memory_key="chat_history", return_messages=True)
        agent = initialize_agent(
            tools, llm, 
            agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
            verbose=True,memory=memory
        )


        initial_message = "You are an AI assistant that can provide helpful answers using available tools.\nIf you are unable to answer, you can use the following tools: GoogleFinance,AlphaVantage, SerpAPI and DuckDuckGo Search."
        memory.chat_memory.add_message(SystemMessage(content=initial_message))
        if message == "exit":
            return

        # Add the user's message to the conversation memory
        memory.chat_memory.add_message(HumanMessage(content=message))
    except Exception as e:
        print(f"An error occurred: {e}")
        return f"An error occurred: {e}"
    # Invoke the agent with the user input and the current chat history
    try:
        response = agent.run(message)
        print("Bot:", response)
        # Add the agent's response to the conversation memory
        memory.chat_memory.add_message(AIMessage(content=response))
        return response
    except Exception as e:
        print(f"An error occurred: {e}")
        return f"An error occurred: {e}"



gr.ChatInterface(
    ask_google
).launch()